In [4]:
import pandas as pd
import numpy as np
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler

In [5]:
df_train = pd.read_feather('all_preprocessed_train.fth')
df_test = pd.read_feather('all_preprocessed_test.fth')

In [6]:
df_train.shape, df_test.shape

((1017209, 82), (41088, 79))

In [7]:
set(df_train.columns) - set(df_test.columns)

{'Customers', 'Sales', 'trend_y', 'week_y'}

## Categorical vars

In [9]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

In [10]:
DataFrameSummary(df_train[cat_vars]).summary().loc[['uniques', 'types']]

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Promo2SinceYear,State,Week,Events,Promo_fw,Promo_bw,StateHoliday_fw,StateHoliday_bw,SchoolHoliday_fw,SchoolHoliday_bw
uniques,1115,7,3,12,31,2,25,26,4,3,...,8,12,52,21,6,6,3,3,8,8
types,numeric,numeric,numeric,numeric,numeric,bool,numeric,numeric,categorical,categorical,...,numeric,categorical,numeric,categorical,numeric,numeric,numeric,numeric,numeric,numeric


In [11]:
# Reemplaza nulls por "" en categoricas
for v in cat_vars: df_train.loc[df_train[v].isnull(), v] = ""

In [12]:
for v in cat_vars: df_test.loc[df_test[v].isnull(), v] = ""

# Continuos Vars

In [13]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

In [14]:
# Reemplaza nulls por 0 en continuas
for v in contin_vars: df_train.loc[df_train[v].isnull(), v] = 0
for v in contin_vars: df_test.loc[df_test[v].isnull(), v] = 0

# Normalización en continuas y LabelEncode en Categoricas

In [15]:
from sklearn_pandas import DataFrameMapper

In [16]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [17]:
mapper_cat = DataFrameMapper(cat_maps)
_ = mapper_cat.fit(df_train)

In [14]:
# Assortment es la novena posición en cat_maps
N = 15
print(df_train['Assortment'].values[:N])
print(mapper_cat.transform(df_train)[:, 9][:N])

['a' 'a' 'a' 'c' 'a' 'a' 'c' 'a' 'c' 'a' 'c' 'c' 'a' 'a' 'c']
[0 0 0 2 0 0 2 0 2 0 2 2 0 0 2]


In [18]:
mapper_cont = DataFrameMapper(contin_maps)
_ = mapper_cont.fit(df_train)

In [19]:
N = 10
print(df_train['CompetitionDistance'].values[:N])
print(mapper_cont.transform(df_train)[:, 0][:N])

[ 1270.   570. 14130.   620. 29910.   310. 24000.  7520.  2030.  3160.]
[-0.53772351 -0.62851175  1.13018633 -0.62202688  3.17681286 -0.6622331
  2.41030064  0.27288586 -0.43915341 -0.29259523]


In [20]:
# La hacemos con todas ahora
df_train[cat_vars] = mapper_cat.transform(df_train)
df_test[cat_vars] = mapper_cat.transform(df_test)

In [21]:
df_train[contin_vars] = mapper_cont.transform(df_train)
df_test[contin_vars] = mapper_cont.transform(df_test)

In [22]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,0,4,2015-07-31,5263,555,1,1.273237,0,2.144211,2,...,0.644376,1.072424,0,0,5,0,5,1,0,1
1,1,4,2015-07-31,6064,625,1,1.273237,0,2.144211,2,...,0.965073,1.072424,0,0,5,0,5,1,0,1
2,2,4,2015-07-31,8314,821,1,1.273237,0,2.144211,2,...,0.644376,1.072424,0,0,5,0,5,1,0,1
3,3,4,2015-07-31,13995,1498,1,1.273237,0,2.144211,2,...,0.965073,1.072424,0,0,5,0,5,1,0,1
4,4,4,2015-07-31,4822,559,1,1.273237,0,2.144211,2,...,0.644376,1.072424,0,0,5,0,5,1,0,1


In [28]:
DataFrameSummary(df_train[cat_vars]).summary().loc[['uniques', 'types']]

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Promo2SinceYear,State,Week,Events,Promo_fw,Promo_bw,StateHoliday_fw,StateHoliday_bw,SchoolHoliday_fw,SchoolHoliday_bw
uniques,1115,7,3,12,31,2,25,26,4,3,...,8,12,52,22,6,6,3,3,8,8
types,numeric,numeric,numeric,numeric,numeric,bool,numeric,numeric,numeric,numeric,...,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric


# Final processing

In [24]:
df_train.shape

(1017209, 82)

In [25]:
df_sales = df_train[df_train.Sales!=0]

In [28]:
df_sales.shape

(844338, 83)

In [31]:
df_test.reset_index(inplace=True)

In [27]:
df_sales.reset_index(inplace=True)

In [38]:
df_sales.to_feather('train_normalized_data.fth')

In [39]:
df_test.to_feather('test_normalized_data.fth')

In [44]:
(df_test['Open'] == 0).sum()

5984

In [45]:
(df_sales['Open'] == 0).sum()

0